The overall goal is to create a Python script that makes a custom model for emotional recognition and deploys it live to a camera.
- The structure of the data is a subset of folders containing images. The folders are labeled by their respective motions: Ahegao, Angry, Happy, Neutral, Sad, Surprise. 
- Convert the RGB images into a numpy array.
- When creating the CNN architecture, add Ridge Regression regularization to each layer. Add Dropout and BatchNormalization layers.
- When training the CNN add Kfold cross validation.
- Make sure the script is using an Nvidia GPU
- Save the best performing checkpoint and label the model as “emotional_recognition_val_accuracy_{val_accuracy:.2f}”
- Then write a script in Python, referring to the prior script, that pre processes the live feed of the 

**More Details Here**
https://chat.openai.com/share/6aca2c47-efd4-41a6-83cd-9eabe7b2d4fd

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
#Best hyperparameters: {'batch_size': 16, 'dropout_val': 0.3, 'epochs': 10, 'l2_val': 0.001, 'num_folds': 10}

In [2]:
import os
import numpy as np
import cv2
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid

#Parameters
num_classes = 6
img_width, img_height = 64, 64 #Resize all images to this size. Ensure minimum resolution is not less than this.
min_res = (img_width, img_height) # Update to set minimum resolution of photos to train on 
max_res = (512, 512) # Update to set maximum resolution of photos to train on 

def load_data(data_dir, min_resolution=None, max_resolution=None):
    images = []
    labels = []
    emotions = ["Ahegao", "Angry", "Happy", "Neutral", "Sad", "Surprise"]
    
    for idx, emotion in enumerate(emotions):
        emotion_folder = os.path.join(data_dir, emotion)
        for img_file in os.listdir(emotion_folder):
            img_path = os.path.join(emotion_folder, img_file)
            image = cv2.imread(img_path)
            height, width, _ = image.shape
            #img = load_img(img_path, color_mode='rgb', target_size=(128, 128))  # Load image in RGB
            #img_gray = cv2.cvtColor(img_to_array(img), cv2.COLOR_RGB2GRAY)  # Convert to grayscale
            #img_rgb_gray = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2RGB)  # Convert back to RGB
            
            # Check image resolution
            if (min_resolution is None or (height >= min_resolution[0] and width >= min_resolution[1])) and \
               (max_resolution is None or (height <= max_resolution[0] and width <= max_resolution[1])):
                image = cv2.resize(image, (img_width, img_height))
                images.append(image)
                labels.append(idx)
    return np.array(images), np.array(labels)




# Load and preprocess data
data_directory = r'C:\Important Files\Python Projects\Facial Emotion Recognition\dataset'  # Replace this with your dataset directory
X, y = load_data(data_directory, min_resolution=min_res, max_resolution=max_res)
X = X / 255.0  # Normalize pixel values

# Convert labels to categorical
y = to_categorical(y, num_classes=num_classes)  # 6 classes for the 6 emotions

# Define the hyperparameters grid for grid search
#Best hyperparameters: {'batch_size': 16, 'dropout_val': 0.3, 'epochs': 5, 'l2_val': 0.01, 'num_folds': 10}
param_grid = {
    'batch_size': [16, 32, 64],
    'l2_val': [0.1, 0.01, 0.001],
    'dropout_val': [0.7, 0.5, 0.3],
    'epochs': [5, 10, 15],
    'num_folds': [2, 5, 10]
}

# Define the CNN model with Ridge regularization
def create_model(l2_val, dropout_val):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3), kernel_regularizer=l2(l2_val)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(l2_val)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_regularizer=l2(l2_val)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_val))
    model.add(Dense(6, activation='softmax'))  # Output layer with 6 classes
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

best_val_acc = 0.0
best_epoch = 0
best_params = {}

# Perform grid search
for params in ParameterGrid(param_grid):
    print(f"Training with hyperparameters: {params}")
    
    batch_size = params['batch_size']
    l2_val = params['l2_val']
    dropout_val = params['dropout_val']
    epochs = params['epochs']
    num_folds = params['num_folds']
    
    # Perform K-fold cross-validation
    kfold = KFold(n_splits=num_folds, shuffle=True)
    fold_no = 1
    
    for train_idx, val_idx in kfold.split(X, y):
        model = create_model(l2_val, dropout_val)
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        checkpoint_path ='emotional_CNN_val_accuracy_{val_accuracy:.2f}.h5'
        checkpoint = ModelCheckpoint(checkpoint_path,
                                     monitor='val_accuracy', verbose=1,
                                     save_best_only=True, mode='max')
        
        history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                            validation_data=(X_val, y_val), callbacks=[checkpoint], verbose=1)
        
        val_acc = max(history.history['val_accuracy'])
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_epoch = np.argmax(history.history['val_accuracy']) + 1
            best_params = params
    
        fold_no += 1

print(f"Best validation accuracy: {best_val_acc} at epoch {best_epoch}")
print("Best hyperparameters:", best_params)



# Now you can deploy the best model to recognize emotions from your camera
# Load the best model using model = load_model('best_model.h5') and perform predictions on live camera feed
# Remember to preprocess the camera feed similar to how the training data was preprocessed

Training with hyperparameters: {'batch_size': 16, 'dropout_val': 0.7, 'epochs': 5, 'l2_val': 0.1, 'num_folds': 2}
Epoch 1/5
337/337 [==============================] - 5s 7ms/step - loss: 4.3623 - accuracy: 0.2734 - val_loss: 1.9217 - val_accuracy: 0.2490

Epoch 00001: val_accuracy improved from -inf to 0.24903, saving model to emotional_CNN_val_accuracy_0.25.h5
Epoch 2/5
337/337 [==============================] - 2s 6ms/step - loss: 1.7227 - accuracy: 0.3379 - val_loss: 1.7790 - val_accuracy: 0.2768

Epoch 00002: val_accuracy improved from 0.24903 to 0.27684, saving model to emotional_CNN_val_accuracy_0.28.h5
Epoch 3/5
337/337 [==============================] - 2s 6ms/step - loss: 1.6613 - accuracy: 0.3576 - val_loss: 1.8009 - val_accuracy: 0.2540

Epoch 00003: val_accuracy did not improve from 0.27684
Epoch 4/5
337/337 [==============================] - 2s 6ms/step - loss: 1.6595 - accuracy: 0.3704 - val_loss: 1.5467 - val_accuracy: 0.4293

Epoch 00004: val_accuracy improved from 0.27

Epoch 1/5
607/607 [==============================] - 4s 6ms/step - loss: 3.0892 - accuracy: 0.3041 - val_loss: 1.8123 - val_accuracy: 0.2651

Epoch 00001: val_accuracy improved from -inf to 0.26506, saving model to emotional_CNN_val_accuracy_0.27.h5
Epoch 2/5
607/607 [==============================] - 3s 5ms/step - loss: 1.6819 - accuracy: 0.3395 - val_loss: 1.6243 - val_accuracy: 0.3791

Epoch 00002: val_accuracy improved from 0.26506 to 0.37905, saving model to emotional_CNN_val_accuracy_0.38.h5
Epoch 3/5
607/607 [==============================] - 3s 5ms/step - loss: 1.6588 - accuracy: 0.3549 - val_loss: 1.8652 - val_accuracy: 0.2929

Epoch 00003: val_accuracy did not improve from 0.37905
Epoch 4/5
607/607 [==============================] - 3s 5ms/step - loss: 1.6352 - accuracy: 0.3745 - val_loss: 1.6884 - val_accuracy: 0.3272

Epoch 00004: val_accuracy did not improve from 0.37905
Epoch 5/5
607/607 [==============================] - 3s 5ms/step - loss: 1.6024 - accuracy: 0.3718 - va

607/607 [==============================] - 3s 5ms/step - loss: 1.6823 - accuracy: 0.3690 - val_loss: 2.2354 - val_accuracy: 0.2764

Epoch 00002: val_accuracy improved from 0.26067 to 0.27644, saving model to emotional_CNN_val_accuracy_0.28.h5
Epoch 3/5
607/607 [==============================] - 3s 5ms/step - loss: 1.6540 - accuracy: 0.3849 - val_loss: 1.5954 - val_accuracy: 0.4091

Epoch 00003: val_accuracy improved from 0.27644 to 0.40909, saving model to emotional_CNN_val_accuracy_0.41.h5
Epoch 4/5
607/607 [==============================] - 3s 5ms/step - loss: 1.6080 - accuracy: 0.3860 - val_loss: 1.7569 - val_accuracy: 0.2913

Epoch 00004: val_accuracy did not improve from 0.40909
Epoch 5/5
607/607 [==============================] - 3s 5ms/step - loss: 1.5863 - accuracy: 0.4002 - val_loss: 1.6082 - val_accuracy: 0.4072

Epoch 00005: val_accuracy did not improve from 0.40909
Epoch 1/5
607/607 [==============================] - 3s 5ms/step - loss: 3.1315 - accuracy: 0.3144 - val_loss:


Epoch 00001: val_accuracy improved from -inf to 0.09824, saving model to emotional_CNN_val_accuracy_0.10.h5
Epoch 2/5
607/607 [==============================] - 3s 5ms/step - loss: 1.6703 - accuracy: 0.4202 - val_loss: 3.1527 - val_accuracy: 0.1909

Epoch 00002: val_accuracy improved from 0.09824 to 0.19092, saving model to emotional_CNN_val_accuracy_0.19.h5
Epoch 3/5
607/607 [==============================] - 3s 5ms/step - loss: 1.5952 - accuracy: 0.4346 - val_loss: 1.7016 - val_accuracy: 0.4041

Epoch 00003: val_accuracy improved from 0.19092 to 0.40408, saving model to emotional_CNN_val_accuracy_0.40.h5
Epoch 4/5
607/607 [==============================] - 3s 5ms/step - loss: 1.5301 - accuracy: 0.4502 - val_loss: 1.4067 - val_accuracy: 0.5190

Epoch 00004: val_accuracy improved from 0.40408 to 0.51900, saving model to emotional_CNN_val_accuracy_0.52.h5
Epoch 5/5
607/607 [==============================] - 3s 5ms/step - loss: 1.4802 - accuracy: 0.4754 - val_loss: 1.6863 - val_accuracy

Epoch 1/5
338/338 [==============================] - 3s 6ms/step - loss: 2.1392 - accuracy: 0.3195 - val_loss: 2.0756 - val_accuracy: 0.3399

Epoch 00001: val_accuracy improved from -inf to 0.33995, saving model to emotional_CNN_val_accuracy_0.34.h5
Epoch 2/5
338/338 [==============================] - 2s 6ms/step - loss: 1.6086 - accuracy: 0.4523 - val_loss: 1.7372 - val_accuracy: 0.4312

Epoch 00002: val_accuracy improved from 0.33995 to 0.43119, saving model to emotional_CNN_val_accuracy_0.43.h5
Epoch 3/5
338/338 [==============================] - 2s 6ms/step - loss: 1.4621 - accuracy: 0.4962 - val_loss: 1.5664 - val_accuracy: 0.4477

Epoch 00003: val_accuracy improved from 0.43119 to 0.44770, saving model to emotional_CNN_val_accuracy_0.45.h5
Epoch 4/5
338/338 [==============================] - 2s 6ms/step - loss: 1.3796 - accuracy: 0.5366 - val_loss: 1.5563 - val_accuracy: 0.4603

Epoch 00004: val_accuracy improved from 0.44770 to 0.46031, saving model to emotional_CNN_val_accuracy

...

152/152 [==============================] - 1s 8ms/step - loss: 1.4528 - accuracy: 0.5804 - val_loss: 1.7214 - val_accuracy: 0.3386

Epoch 00002: val_accuracy did not improve from 0.43878
Epoch 3/15
152/152 [==============================] - 1s 8ms/step - loss: 1.3092 - accuracy: 0.6113 - val_loss: 1.4754 - val_accuracy: 0.5074

Epoch 00003: val_accuracy improved from 0.43878 to 0.50742, saving model to emotional_CNN_val_accuracy_0.51.h5
Epoch 4/15
152/152 [==============================] - 1s 8ms/step - loss: 1.2224 - accuracy: 0.6407 - val_loss: 1.8241 - val_accuracy: 0.3785

Epoch 00004: val_accuracy did not improve from 0.50742
Epoch 5/15
152/152 [==============================] - 1s 8ms/step - loss: 1.1838 - accuracy: 0.6630 - val_loss: 1.2443 - val_accuracy: 0.6271

Epoch 00005: val_accuracy improved from 0.50742 to 0.62709, saving model to emotional_CNN_val_accuracy_0.63.h5
Epoch 6/15
152/152 [==============================] - 1s 8ms/step - loss: 1.1445 - accuracy: 0.6688 - val_l

135/135 [==============================] - 1s 8ms/step - loss: 1.0182 - accuracy: 0.6830 - val_loss: 1.3606 - val_accuracy: 0.5012

Epoch 00003: val_accuracy did not improve from 0.50626
Epoch 4/15
135/135 [==============================] - 1s 8ms/step - loss: 0.8843 - accuracy: 0.7333 - val_loss: 1.6125 - val_accuracy: 0.4645

Epoch 00004: val_accuracy did not improve from 0.50626
Epoch 5/15
135/135 [==============================] - 1s 8ms/step - loss: 0.7887 - accuracy: 0.7813 - val_loss: 1.1574 - val_accuracy: 0.6198

Epoch 00005: val_accuracy improved from 0.50626 to 0.61984, saving model to emotional_CNN_val_accuracy_0.62.h5
Epoch 6/15
135/135 [==============================] - 1s 8ms/step - loss: 0.7165 - accuracy: 0.8166 - val_loss: 1.7013 - val_accuracy: 0.5206

Epoch 00006: val_accuracy did not improve from 0.61984
Epoch 7/15
135/135 [==============================] - 1s 8ms/step - loss: 0.6768 - accuracy: 0.8307 - val_loss: 1.5590 - val_accuracy: 0.5600

Epoch 00007: val_acc

Epoch 3/15
152/152 [==============================] - 1s 8ms/step - loss: 1.0301 - accuracy: 0.6690 - val_loss: 1.6142 - val_accuracy: 0.3976

Epoch 00003: val_accuracy did not improve from 0.44022
Epoch 4/15
152/152 [==============================] - 1s 8ms/step - loss: 0.9196 - accuracy: 0.7140 - val_loss: 1.9784 - val_accuracy: 0.3670

Epoch 00004: val_accuracy did not improve from 0.44022
Epoch 5/15
152/152 [==============================] - 1s 8ms/step - loss: 0.8345 - accuracy: 0.7572 - val_loss: 1.4091 - val_accuracy: 0.5709

Epoch 00005: val_accuracy improved from 0.44022 to 0.57090, saving model to emotional_CNN_val_accuracy_0.57.h5
Epoch 6/15
152/152 [==============================] - 1s 8ms/step - loss: 0.7698 - accuracy: 0.7860 - val_loss: 1.9213 - val_accuracy: 0.4829

Epoch 00006: val_accuracy did not improve from 0.57090
Epoch 7/15
152/152 [==============================] - 1s 8ms/step - loss: 0.7132 - accuracy: 0.8098 - val_loss: 1.5398 - val_accuracy: 0.5941

Epoch 000

Epoch 3/15
152/152 [==============================] - 1s 8ms/step - loss: 1.0505 - accuracy: 0.6582 - val_loss: 1.2818 - val_accuracy: 0.5714

Epoch 00003: val_accuracy improved from 0.50278 to 0.57143, saving model to emotional_CNN_val_accuracy_0.57.h5
Epoch 4/15
152/152 [==============================] - 1s 8ms/step - loss: 0.9445 - accuracy: 0.6988 - val_loss: 1.1680 - val_accuracy: 0.6076

Epoch 00004: val_accuracy improved from 0.57143 to 0.60761, saving model to emotional_CNN_val_accuracy_0.61.h5
Epoch 5/15
152/152 [==============================] - 1s 8ms/step - loss: 0.8640 - accuracy: 0.7353 - val_loss: 1.5569 - val_accuracy: 0.5455

Epoch 00005: val_accuracy did not improve from 0.60761
Epoch 6/15
152/152 [==============================] - 1s 8ms/step - loss: 0.8132 - accuracy: 0.7598 - val_loss: 1.6750 - val_accuracy: 0.4740

Epoch 00006: val_accuracy did not improve from 0.60761
Epoch 7/15
152/152 [==============================] - 1s 8ms/step - loss: 0.7731 - accuracy: 0.7


Epoch 00009: val_accuracy did not improve from 0.58627
Epoch 10/15
152/152 [==============================] - 1s 8ms/step - loss: 0.6140 - accuracy: 0.8613 - val_loss: 1.8217 - val_accuracy: 0.5705

Epoch 00010: val_accuracy did not improve from 0.58627
Epoch 11/15
152/152 [==============================] - 1s 8ms/step - loss: 0.6012 - accuracy: 0.8714 - val_loss: 1.6383 - val_accuracy: 0.5946

Epoch 00011: val_accuracy improved from 0.58627 to 0.59462, saving model to emotional_CNN_val_accuracy_0.59.h5
Epoch 12/15
152/152 [==============================] - 1s 8ms/step - loss: 0.5754 - accuracy: 0.8842 - val_loss: 1.7865 - val_accuracy: 0.5761

Epoch 00012: val_accuracy did not improve from 0.59462
Epoch 13/15
152/152 [==============================] - 1s 8ms/step - loss: 0.5667 - accuracy: 0.8834 - val_loss: 2.0367 - val_accuracy: 0.5779

Epoch 00013: val_accuracy did not improve from 0.59462
Epoch 14/15
152/152 [==============================] - 1s 8ms/step - loss: 0.5416 - accuracy

In [2]:
import os
import numpy as np
import cv2
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
img_width, img_height = 64, 64 

In [4]:
# Live feed preprocessing script should be implemented to process the live camera frames using OpenCV
# and feed them into the trained model for inference
# This involves capturing frames from the camera, preprocessing them similarly to the training data,
# and performing inference using the trained model.
# You can use OpenCV's VideoCapture to access the live camera feed.

# Load the trained model
model = load_model('emotional_CNN_val_accuracy_0.67_medium_traning_sample.h5')  # Load your best model here

# Define emotions for mapping the output
emotions = ["Ahegao", "Angry", "Happy", "Neutral", "Sad", "Surprise"]

# Function to preprocess live feed frames
def preprocess_live_frame(frame):
    # Resize the frame to match the input size of the trained model
    resized_frame = cv2.resize(frame, (img_width, img_height))
    frame_normalized = resized_frame / 255.0  # Normalize pixel values
    return np.expand_dims(frame_normalized, axis=0)

# Access the live camera feed
cap = cv2.VideoCapture(0)  # Use 0 for the default camera, you may need to change this number

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Preprocess the live frame
    processed_frame = preprocess_live_frame(frame)
    
    # Perform inference using the trained model
    predictions = model.predict(processed_frame)
    confidence = np.max(predictions)
    emotion_label = emotions[np.argmax(predictions)]
    
    # Display recognized emotion and confidence level on the camera feed
    text = f"Emotion: {emotion_label}, Confidence: {confidence:.2f}"
    cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    
    # Show the camera feed with emotion and confidence
    cv2.imshow('Emotion Recognition', frame)
    
    # Exit loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close windows
cap.release()
cv2.destroyAllWindows()
